In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os 
os.chdir('/home/kkotzen/research/PPG_sleepstaging_orion2 /')
from pathlib import Path
from src.parsing.MESAParser import MESAParser

from scipy import signal, stats
from scipy.signal import butter, resample, sosfiltfilt

fs = 256
resample_fs = 32
F = fs/resample_fs
highcut = 10
lowcut = 0.00001
order=8

sos = butter(order, [lowcut / (fs / 2), highcut / (fs / 2)], 'bandpass', output='sos')


In [2]:
import time 

dl = MESAParser()
patients = ['0050'] #dl.database_all_patient_IDs
x = dl.load_signal(patients[0], 'Pleth')
x = sosfiltfilt(sos, x, axis=0)
x = resample(x, int(len(x)/F))
fs = 32

In [3]:
r = [0.0001, 0.01, 0.1, 0.15, 0.5, 2, 4]
x_filt = np.empty((len(r), len(x)))
for i in range(len(r)-1):
    sos = butter(order, [r[i] / (fs / 2), r[i+1] / (fs / 2)], 'bandpass', output='sos')
    x_filt[i] = sosfiltfilt(sos, x, axis=0)


In [4]:
plt.close('all')
fig, ax = plt.subplots(2,3, sharex=True)
ax[0,0].plot(x)
for i in range(0,2,1):
    ax[0,i+1].plot(x_filt[i])
    ax[0,i+1].set_ylabel(f"{r[i]} to {r[i+1]}")
for i in range(0,3,1):
    ax[1,i].plot(x_filt[i+2])
    ax[1,i].set_ylabel(f"{r[i+2]} to {r[i+3]}")
    
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def _time_series_subsequences(ts, window, hop=1):
    assert len(ts.shape) == 1
    shape = (int(int(ts.size - window) / hop + 1), window)
    strides = ts.strides[0] * hop, ts.strides[0]
    return np.lib.stride_tricks.as_strided(ts, shape=shape, strides=strides)

X = _time_series_subsequences(x_32, 32*30, 32*30)
print(X.shape)

NameError: name 'x_32' is not defined

In [ ]:
plt.close('all')
plt.plot(X[100])

In [5]:
plt.close('all')
plt.plot(X[100])
for i in [2,4,8,16,30]:
    win, hop = i, i/2
    f, t, Sxx = signal.spectrogram(X[100], fs, nperseg = fs*win, noverlap=fs*hop, window='tukey')
    plt.figure(figsize=(4,4))
    plt.title(f"{win}s window with {hop}s overlap. {Sxx.shape}")
    if Sxx.shape[1] > 1:
        plt.pcolormesh(t, f, np.log(Sxx))
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [sec]')
    else:
        plt.plot(f, (np.mean(Sxx, axis=1)*100))
        plt.ylabel('Amplitude')
        plt.xlabel('Frequency [Hz]')
#     plt.show()


NameError: name 'X' is not defined

In [9]:
plt.close('all')
widths = np.arange(1, 31)
cwtmatr = signal.cwt(x_32, signal.ricker, widths)
# X_ = X[0:10]
# cwtmatr = np.zeros((30, X_.shape[0]*X_.shape[1]))
# print(cwtmatr.shape)
# for i, x in enumerate(X_):
#     cwt = signal.cwt(x, signal.ricker, widths)
#     cwtmatr[:, i*x.shape[0]:(i+1)*x.shape[0]] =(cwt-np.mean(cwt))/np.std(cwt) 

# for i in range(cwtmatr.shape[0]):
#     cwt = cwtmatr[i,:]
#     cwtmatr[i,:] = (cwt-np.mean(cwt))/np.std(cwt) 
# plt.imshow((cwtmatr), cmap='PRGn', aspect='auto',
#            vmax=abs((cwtmatr)).max(), vmin=-abs((cwtmatr)).max(), origin='lower')

# print(cwtmatr.shape)
# plt.imshow(, cmap='PRGn', aspect='auto',
#            vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max(), origin='lower')
# plt.subplot(4,1,3, sharex=ax1)
# plt.plot(cwtmatr[5,:])
# plt.subplot(4,1,4, sharex=ax1)
# plt.plot(cwtmatr[25,:])

# plt.show()
plt.figure()
plt.imshow((cwtmatr[:,3200:3520]), cmap='PRGn', aspect='auto',
           vmax=abs((cwtmatr[:,3200:3520])).max(), vmin=-abs((cwtmatr[:,3200:3520])).max(), origin='lower')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
win, hop = 16, 8
Sxx = np.empty((X.shape[0], 257, 2))
for i in range(X.shape[0]):
    f, t, Sxx_i = signal.spectrogram(X[i], fs, nperseg = fs*win, noverlap=fs*hop, window='hamming')
    Sxx[i] = Sxx_i

In [ ]:
Sxx_mean = np.mean(Sxx[0:900, :, :], axis=2)
plt.close('all')
plt.figure(figsize=(8,4))
plt.pcolormesh(np.arange(900), f[0:60], (Sxx_mean.T)[0:60,:])

In [ ]:
import numpy as np
from numpy.lib.stride_tricks import as_strided

def pool1d(ts, window, hop, padding, pool_mode='max'):
    assert len(ts.shape) == 1
    shape = (int(int(ts.size - window) / hop + 1), window)
    strides = ts.strides[0] * hop, ts.strides[0]
    strided = np.lib.stride_tricks.as_strided(ts, shape=shape, strides=strides)
    if pool_mode=='max':
        return np.max(strided, axis=1).reshape(-1)
    else:
        return np.mean(strided, axis=1).reshape(-1)


In [ ]:
import copy
x_32 = copy.deepcopy(X[100])

In [ ]:
x_16_max = pool1d(x_32, 2, 2,0, 'max')
x_16_ave = pool1d(x_32, 2, 2,0, 'ave')
x_8_max = pool1d(x_16_max, 2, 2,0, 'max')
x_8_ave = pool1d(x_16_ave, 2, 2,0, 'ave')
x_4_max = pool1d(x_8_max, 2, 2,0, 'max')
x_4_ave = pool1d(x_8_ave, 2, 2,0, 'ave')
x_2_max = pool1d(x_4_max, 2, 2,0, 'max')
x_2_ave = pool1d(x_4_ave, 2, 2,0, 'ave')
x_1_max = pool1d(x_2_max, 2, 2,0, 'max')
x_1_ave = pool1d(x_2_ave, 2, 2,0, 'ave')

In [ ]:
plt.close('all')
t_32 = np.arange(0, len(x_32)/32, 1/32)
t_16 = np.arange(0, len(x_16_max)/16, 1/16)
t_8 = np.arange(0, len(x_8_max)/8, 1/8)
t_4 = np.arange(0, len(x_4_max)/4, 1/4)
t_2 = np.arange(0, len(x_2_max)/2, 1/2)
t_1 = np.arange(0, len(x_1_max)/1, 1/1)

plt.plot(t_32, x_32, label='x32')
plt.plot(t_16, x_16_max, label='x16max')
# plt.plot(t_16, x_16_ave, label='x16ave')
plt.plot(t_8, x_8_max, label='x8max')
# plt.plot(t_8, x_8_ave, label='x8ave')
plt.plot(t_4, x_4_max, label='x4max')
# plt.plot(t_4, x_4_ave, label='x4ave')
plt.plot(t_2, x_2_max, label='x2max')
# plt.plot(t_2, x_2_ave, label='x2ave')
plt.plot(t_1, x_1_max, label='x1max')
# plt.plot(t_1, x_1_ave, label='x1ave')
plt.legend(loc='right')


In [ ]:
plt.close('all')
t_32 = np.arange(0, len(x_32)/32, 1/32)
t_16 = np.arange(0, len(x_16_max)/16, 1/16)
t_8 = np.arange(0, len(x_8_max)/8, 1/8)
t_4 = np.arange(0, len(x_4_max)/4, 1/4)
t_2 = np.arange(0, len(x_2_max)/2, 1/2)
t_1 = np.arange(0, len(x_1_max)/1, 1/1)

plt.plot(t_32, x_32, label='x32')
plt.plot(t_16, x_16_ave, label='x16ave')
plt.plot(t_8, x_8_ave, label='x8ave')
plt.plot(t_4, x_4_ave, label='x4ave')
plt.plot(t_2, x_2_ave, label='x2ave')
plt.plot(t_1, x_1_ave, label='x1ave')
plt.legend(loc='right')
